# Laboratorium 7

## Import zależności

In [1]:
import importlib

In [2]:
import regex
importlib.reload(regex)
from regex import Regex

## Rozwiązanie

Moja implementacja realizuje funkcję *match* sprawdzającą czy podany tekst jest akceptowany przez wprowadzone wyrażenie regularne. 

Wyrażenie regularne jest kompilowane do postaci niedeterministycznego automatu skończonego za pomocą konstrukcji Thompsona. Dla ułatwienia przed konstrukcją automatu wyrażenie regularne przedstawiane jest w odwróconej notacji polskiej. 

### Implementacja obsługuje:

**Znaki traktowane literalnie**

In [3]:
re = Regex('abcd 100deff_')

In [4]:
text = 'abcd 100deff_'
re.match(text)

True

In [5]:
text = 'abcd 900deff_'
re.match(text)

False

**Znak kropki**

Oznaczający dowolny znak.

In [6]:
re = Regex('a.c.d.f')

In [7]:
text = 'abc dbf'
re.match(text)

True

In [8]:
text = 'a.c.d.f'
re.match(text)

True

In [9]:
text = 'abcccdef'
re.match(text)

False

**Operator gwiazdki**

Oznaczający zero lub więcej powtórzeń poprzedzającego wyrażenia. 

In [10]:
re = Regex('a.c*d.f')

In [11]:
text = 'abcccdef'
re.match(text)

True

In [12]:
text = 'abdef'
re.match(text)

True

In [13]:
text = 'abcdef'
re.match(text)

True

In [14]:
text = 'abddef'
re.match(text)

False

**Operator plusa**

Oznaczający jedno lub więcej powtórzeń poprzedniego wyrażenia.

In [15]:
re = Regex('a.c+d.f')

In [16]:
text = 'abcccdef'
re.match(text)

True

In [17]:
text = 'abdef'
re.match(text)

False

In [18]:
text = 'abcdef'
re.match(text)

True

In [19]:
text = 'abddef'
re.match(text)

False

**Operator pytajnika**

Oznaczający, że poprzednie wyrażenie wystąpi dokładnie raz, lub nie wystąpi w ogóle. 

In [20]:
re = Regex('a.c?d.f')

In [21]:
text = 'abcccdef'
re.match(text)

False

In [22]:
text = 'abdef'
re.match(text)

True

In [23]:
text = 'abcdef'
re.match(text)

True

In [24]:
text = 'abddef'
re.match(text)

False

**Nawiasy**

Operatory mogą być aplikowane względem wyrażeń ujętych w nawiasach. 

In [25]:
re = Regex('a(c.d)*f')

In [26]:
text = 'acddcadf'
re.match(text)

True

In [27]:
text = 'acddcadcf'
re.match(text)

False

In [28]:
text = 'a(c.d)*f'
re.match(text)

False

Nawiasy obsługują również zagnieżdżenia. 

In [29]:
re = Regex('a(f(bc)+e)*d')

In [30]:
text = 'afbcbcbcbcbcefbcbcbcbcbced'
re.match(text)

True

In [31]:
text = 'afefbcbcbcbcbced'
re.match(text)

False

**Klasy znaków**

Wyrażenie może zawierać grupy znaków wyrażane pomiędzy nawiasami kwadratowymi. 

In [32]:
re = Regex('ab[cdzf-h]xyz')

In [33]:
text = 'abgxyz'
re.match(text)

True

In [34]:
text = 'abcdxyz'
re.match(text)

False

In [35]:
text = 'abcxyz'
re.match(text)

True

In [36]:
text = 'ab[cdzf-h]xyz'
re.match(text)

False

In [37]:
text = 'abxyz'
re.match(text)

False

In [38]:
re = Regex('ab[cdzf-h]*xyz')

In [39]:
text = 'abcdhxyz'
re.match(text)

True

In [40]:
text = 'abixyz'
re.match(text)

False

Może też zawierać predefiniowane grupy znaków: 
* \d - cyfra
* \s - znak biały
* \c - znak alfabetyczny (to jest [a-zA-Z])
* \w - znak alfabetyczny, lub alfanumeryczny (to jest [a-zA-Z0-9])

In [41]:
re = Regex('ab\dxyz')

In [42]:
text = 'ab9xyz'
re.match(text)

True

In [43]:
text = 'abdxyz'
re.match(text)

False

In [44]:
re = Regex('ab\d*xyz')

In [45]:
text = 'ab0912xyz'
re.match(text)

True

In [46]:
re = Regex('ab\sxyz')

In [47]:
text = 'ab xyz'
re.match(text)

True

In [48]:
text = 'absxyz'
re.match(text)

False

In [49]:
re = Regex('ab\cxyz')

In [50]:
text = 'abbxyz'
re.match(text)

True

In [51]:
text = 'ab xyz'
re.match(text)

False

In [52]:
text = 'ab0xyz'
re.match(text)

False

In [53]:
re = Regex('ab\wxyz')

In [54]:
text = 'abbxyz'
re.match(text)

True

In [55]:
text = 'ab xyz'
re.match(text)

False

In [56]:
text = 'ab0xyz'
re.match(text)

True

### Sposób przetwarzania wyrażenia regularnego

W pierwszym kroku do wyrażenia dodawany jest znak konkatenacji - łączy kolejne wyrażenia. 

In [57]:
re = Regex('abd', verbose=True)

Regex: abd
Preprocessed regex: a^b^d
Regex in reversed polish notation: ab^d^


Następnie grupy znaków przetwarzane są do postaci wyrażenia łączonego operatorem | - tak, by można je było przedstawić w odwrotnej notacji polskiej. 

In [58]:
re = Regex('a[bc]d', verbose=True)

Regex: a[bc]d
Preprocessed regex: a^(b|c)^d
Regex in reversed polish notation: abc|^d^


Tak samo przedstawione zostają grupy predefiniowane. 

In [59]:
re = Regex('a\dd', verbose=True)

Regex: a\dd
Preprocessed regex: a^(1|2|3|4|5|6|7|8|0|9)^d
Regex in reversed polish notation: a12|3|4|5|6|7|8|0|9|^d^


In [60]:
re = Regex('a\wd', verbose=True)

Regex: a\wd
Preprocessed regex: a^(b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|1|2|3|4|5|6|7|8|a|z|A|Z|0|9)^d
Regex in reversed polish notation: abc|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|1|2|3|4|5|6|7|8|a|z|A|Z|0|9|^d^


A także grupy podane w formie zakresu.

In [61]:
re = Regex('a[A-D]d', verbose=True)

Regex: a[A-D]d
Preprocessed regex: a^(A|B|C|D)^d
Regex in reversed polish notation: aAB|C|D|^d^


W kolejnym kroku tak przygotowane wyrażenie zostaje przekonwertowane do odwrotnej notacji polskiej za pomocą algorytmu stacji rozrządowej, w wersji iteracyjnej z wykorzystaniem stosu. Na wyjściu otrzymujemy oczywiście wyrażenie bez nawiasów - zgodnie z zasadami ONP. 

W efekcie otrzymujemy kompletne wyrażenie regularne w postaci ONP. 

In [62]:
re = Regex('a(b(fg)*cd)+d', verbose=True)

Regex: a(b(fg)*cd)+d
Preprocessed regex: a^(b^(f^g)*^c^d)+^d
Regex in reversed polish notation: abfg^*^c^d^+^d^


Na podstawie wyrażenia w ONP konstruowany jest niedeterministyczny automat skończony. Do jego utworzenia wykorzystywane są operacje rozszerzenia bazowych automatów. 
* Operacja konkatenacji
![Konkatenacja](img/concat.png)
* Operacja unii
![Unia](img/unia.png)
* Domknięcie Kleene’ego
![Unia](img/closure.png)
* Operacja + (modyfikacja domknięcia)
![Unia](img/one_or_more.png)
* Operacja ? (modyfikacja domknięcia)
![Unia](img/maybe.png)

Wyrażenie ONP przetwarzamy korzystając ze stosu, na którym trzymamy utworzone do tej pory automaty. Gdy podczas przetwarzania wyrażenia napotkamy na operator, aplikujemy go do automatów ze szczytu stosu. Podczas tworzenia automatu operator '.' traktujemy tak samo, jak symbole literalne, tj umieszczając na szczycie stosu nowy automat jedynie z tym symbolem. 

### Test dla złożonych wyrażeń

In [63]:
re = Regex('ab*\cd.z')

In [64]:
text = 'abbbbbcddz'
re.match(text)

True

In [65]:
re = Regex('a.*b')

In [66]:
text = 'acdefghijklmnb'
re.match(text)

True

In [67]:
text = 'acdefghijklmn'
re.match(text)

False

In [68]:
re = Regex('a(b(cl)*d)*(e[fijk]?g)+h')

In [69]:
text = 'abcldefgh'
re.match(text)

True

In [70]:
text = 'abclclclcldbcldbdbclclclclclcldefgeigejgegh'
re.match(text)

True

## Literatura
*  Alfred Vaino Aho, Monica S. Lam, Ravi Sethi, Jeffrey D. Ullman, *3.7.4 Construction of an NFA from a Regular Expression*, *Compilers : Principles, Techniques, & Tools (2nd ed.)*, Boston, MA, USA: Pearson Addison-Wesley. p. 159. ISBN 9780321486813 (2007)
* Ken Thompson, *Programming Techniques: Regular expression search algorithm*, (Jun 1968) Communications of the ACM. 11 (6): 419–422, [link](https://www.fing.edu.uy/inco/cursos/intropln/material/p419-thompson.pdf) 
* *Shunting-yard algorithm (by Edsger Dijkstra)*, Wikipedia, [link](https://en.wikipedia.org/wiki/Shunting-yard_algorithm), last access: 20.06.2020
* Denis Kyashif, *Implementing a Regular Expression Engine*, [link](https://deniskyashif.com/2019/02/17/implementing-a-regular-expression-engine/), last access: 20.06.2020
* *Thompson's construction*, Wikipedia, [link](https://en.wikipedia.org/wiki/Thompson%27s_construction), last access: 20.06.2020

